# Spark on Tour
## Ejemplo de procesamiento de datos en streaming para generar un dashboard en NRT

En este notebook vamos a ver un ejemplo completo de como se podría utilizar la API de streaming estructurado de Spark para procesar un stream de eventos de puntuación en vivo, en el tiempo real, y generar como salida un conjunto de estadísticas, o valores agregados, con los que poder construir un dashboard de visualización y monitorización en tiempo real.

Particularmente vamos a simular una plataforma de vídeo bajo demanda en la que los usuarios están viendo pelítculas y puntuándolas. Tomaremos los eventos de puntuación que van entrando en streaming, y genrar, en tiempo real, estadísticas de visualización agredas por género, de forma que podamos monitorizar qué géneros de películas son los más populares en este momento.


### Importamos librerías, definimos esquemas e inicializamos la sesión Spark.

In [9]:
import findspark
findspark.init()

import pyspark
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from IPython.display import clear_output
import plotly.express as px

In [10]:
ratingSchema = StructType([
    StructField("user", IntegerType()),
    StructField("movie", IntegerType()),
    StructField("rating", FloatType())
])

movieSchema = StructType([
    StructField("movie", IntegerType()),
    StructField("title", StringType()),
    StructField("genres", StringType())
])

In [11]:
sparkSession = (SparkSession.builder
                .appName("Movie ratings streaming")
                .master("local[*]")
                .config("spark.scheduler.mode", "FAIR")
                .getOrCreate())
sparkSession.sparkContext.setLogLevel("ERROR")

### Leemos el dataset de películas

In [13]:
movies = sparkSession.read.csv("/tmp/movielens/movies.csv", schema=movieSchema, header=True)
movies.show()

+-----+--------------------+--------------------+
|movie|               title|              genres|
+-----+--------------------+--------------------+
|    1|    Toy Story (1995)|Adventure|Animati...|
|    2|      Jumanji (1995)|Adventure|Childre...|
|    3|Grumpier Old Men ...|      Comedy|Romance|
|    4|Waiting to Exhale...|Comedy|Drama|Romance|
|    5|Father of the Bri...|              Comedy|
|    6|         Heat (1995)|Action|Crime|Thri...|
|    7|      Sabrina (1995)|      Comedy|Romance|
|    8| Tom and Huck (1995)|  Adventure|Children|
|    9| Sudden Death (1995)|              Action|
|   10|    GoldenEye (1995)|Action|Adventure|...|
|   11|American Presiden...|Comedy|Drama|Romance|
|   12|Dracula: Dead and...|       Comedy|Horror|
|   13|        Balto (1995)|Adventure|Animati...|
|   14|        Nixon (1995)|               Drama|
|   15|Cutthroat Island ...|Action|Adventure|...|
|   16|       Casino (1995)|         Crime|Drama|
|   17|Sense and Sensibi...|       Drama|Romance|


### Transformamos el dataset de películas para asociar cada película con cada uno de sus genéros 


In [14]:
movies = movies.select("movie", "title", split("genres", "\|").alias("genres"))
movies.show(10)

+-----+--------------------+--------------------+
|movie|               title|              genres|
+-----+--------------------+--------------------+
|    1|    Toy Story (1995)|[Adventure, Anima...|
|    2|      Jumanji (1995)|[Adventure, Child...|
|    3|Grumpier Old Men ...|   [Comedy, Romance]|
|    4|Waiting to Exhale...|[Comedy, Drama, R...|
|    5|Father of the Bri...|            [Comedy]|
|    6|         Heat (1995)|[Action, Crime, T...|
|    7|      Sabrina (1995)|   [Comedy, Romance]|
|    8| Tom and Huck (1995)|[Adventure, Child...|
|    9| Sudden Death (1995)|            [Action]|
|   10|    GoldenEye (1995)|[Action, Adventur...|
+-----+--------------------+--------------------+
only showing top 10 rows



In [15]:
movies = movies.select("movie", "title", explode("genres").alias("genre"))
movies.show(10)

+-----+--------------------+---------+
|movie|               title|    genre|
+-----+--------------------+---------+
|    1|    Toy Story (1995)|Adventure|
|    1|    Toy Story (1995)|Animation|
|    1|    Toy Story (1995)| Children|
|    1|    Toy Story (1995)|   Comedy|
|    1|    Toy Story (1995)|  Fantasy|
|    2|      Jumanji (1995)|Adventure|
|    2|      Jumanji (1995)| Children|
|    2|      Jumanji (1995)|  Fantasy|
|    3|Grumpier Old Men ...|   Comedy|
|    3|Grumpier Old Men ...|  Romance|
+-----+--------------------+---------+
only showing top 10 rows



### Inicializamos la carga del stream de puntuaciones desde Apache Kafka

In [17]:
dataset = (sparkSession
        .readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "localhost:29092")
        .option("subscribe", "ratings")
        .load())
dataset = dataset.selectExpr("CAST(value AS STRING)")
dataset = dataset.select(from_json(col("value"), ratingSchema).alias("data")).select("data.*")

### Integramos, sobre la marcha, con el dataset de películas para asociar cada puntuación a los géneros de us película

In [18]:
movieDataset = dataset.join(movies, "movie", "left_outer")

### Agrupamos por género y contamos el número de votaciones (películas vistas) y su puntuación media

In [19]:
movieDataset = movieDataset.select("genre", "rating") \
                .groupBy("genre") \
                .agg(count("rating").alias("num_ratings"), avg("rating").alias("avg_rating")) \
                .sort(desc("num_ratings"))

### Procesamos cada micro-batch de salida para mostrar una gráfica
Este es un paso opcional. Para cada micro-batch de salida podemos ejecutar una función, que en este caso vamos a utilizar para mostrar (y mantener actualizada) una gráfica en tiempo real y así montar un dashboard fácilmente.

En caso real este paso podríamos usarlo para ejecutar código propio de escritura en BBDD, o similar.

In [20]:
def foreach_batch_function(df, epoch_id):
    mostPopularGenres = df.limit(10).toPandas()
    clear_output()
    print(mostPopularGenres)
    px.bar(mostPopularGenres, x='genre', y='num_ratings').show()
    px.bar(mostPopularGenres, x='genre', y='avg_rating').show()

### Ejecutamos el procesamiento en streaming

Este es una de las diferencias con la API estructurada en batch, en streaming debemos "arrancar" el stream explícitamente, y en dicho procesos a configurar como queremos que funcionen los micro-batches:

* *outputMode*: Nos permite indicar como queremos que sea el dataset de salida, en este caso estamos indicando que queremos que todos los micro-batches de salida se integren en un dataset final único.
* *format*: Nos permite indicar como queremos hacer la salida, en este caso por consola, pero podría ser de nuevo enviarlo a Kafka o una BBDD.
* *trigger*: Configuramos el trigger de procesamiento de los micro-batches, en este caso cada 5 segundos se generar un batch con los eventos que hayan llegado
* *foreachBatch*: Parámetro opcional que nos permite indicar una función que se ejecutara para cada micro-batch de datos procesado.

In [21]:
query = movieDataset \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .foreachBatch(foreach_batch_function) \
    .start()

In [22]:
query.explain()
query.awaitTermination()

       genre  num_ratings  avg_rating
0      Drama         1166    3.710978
1     Comedy         1013    3.393880
2   Thriller          950    3.635789
3     Action          934    3.481799
4  Adventure          659    3.525038
5    Romance          555    3.563964
6      Crime          539    3.604824
7     Sci-Fi          323    3.393189
8   Children          284    3.559859
9    Fantasy          233    3.442060


KeyboardInterrupt: 